In [71]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [72]:
df = pd.read_csv('outputs/dataset_20210523.csv')
df_champions = pd.read_csv('data/dataset_champions.csv')
df

,game,blueTop,blueTopGP,blueTopWR,blueTopKDA,blueJungle,blueJungleGP,blueJungleWR,blueJungleKDA,blueMid,...,redMidKDA,redAdc,redAdcGP,redAdcWR,redAdcKDA,redSupport,redSupportGP,redSupportWR,redSupportKDA,result
0,ESPORTSTMNT03/1632489,82,8,0.50,2.9,104,27,0.85,5.9,142,...,3.6,81,21,0.57,4.3,89,10,0.60,2.5,0
1,ESPORTSTMNT03/1632500,79,14,0.36,2.6,104,63,0.63,5.1,68,...,4.9,21,10,0.70,5.3,3,11,0.55,2.8,1
2,ESPORTSTMNT03/1632502,79,14,0.36,2.6,104,63,0.63,5.1,142,...,4.4,21,10,0.70,5.3,3,11,0.55,2.8,1
3,6909-9183,43,2,0.00,0.9,80,5,0.60,3.9,134,...,5.2,145,43,0.60,6.3,3,4,0.75,4.5,1
4,6909-9184,24,16,0.56,2.8,120,11,0.55,3.6,61,...,5.2,145,43,0.60,6.3,57,2,0.50,1.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4055,ESPORTSTMNT01/1887455,266,45,0.64,2.9,76,0,0.00,0.0,58,...,4.9,145,36,0.75,5.0,79,14,0.79,3.1,1
4056,ESPORTSTMNT05/1870404,150,7,0.57,2.9,48,16,0.56,4.1,245,...,6.3,18,5,0.60,4.4,12,16,0.69,4.0,1
4057,ESPORTSTMNT04/1600494,150,9,0.89,5.8,120,8,0.88,6.2,34,...,3.4,222,5,0.40,2.5,201,23,0.61,3.6,0
4058,ESPORTSTMNT02/1722049,79,2,0.00,1.1,72,3,0.00,0.8,13,...,6.1,145,15,0.40,4.3,12,14,0.57,3.4,1


In [73]:
y = df['result'].copy()
X = df.drop(['blueTop', 'blueJungle', 'blueMid', 'blueADC', 'blueSupport', 'redTop', 'redJungle', 'redMid', 'redAdc', 'redSupport', 'result', 'game'],axis=1)
X

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,8,0.50,2.9,27,0.85,5.9,59,0.76,10.7,10,...,4.3,13,0.46,3.6,21,0.57,4.3,10,0.60,2.5
1,14,0.36,2.6,63,0.63,5.1,1,0.00,1.0,20,...,4.1,55,0.69,4.9,10,0.70,5.3,11,0.55,2.8
2,14,0.36,2.6,63,0.63,5.1,5,0.40,2.9,2,...,6.4,3,0.67,4.4,10,0.70,5.3,11,0.55,2.8
3,2,0.00,0.9,5,0.60,3.9,36,0.75,6.2,5,...,3.0,6,0.67,5.2,43,0.60,6.3,4,0.75,4.5
4,16,0.56,2.8,11,0.55,3.6,50,0.64,6.1,5,...,2.9,6,0.67,5.2,43,0.60,6.3,2,0.50,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4055,45,0.64,2.9,0,0.00,0.0,3,0.33,2.6,6,...,7.9,55,0.69,4.9,36,0.75,5.0,14,0.79,3.1
4056,7,0.57,2.9,16,0.56,4.1,7,0.71,3.4,8,...,9.4,10,0.70,6.3,5,0.60,4.4,16,0.69,4.0
4057,9,0.89,5.8,8,0.88,6.2,1,1.00,5.0,9,...,4.5,36,0.64,3.4,5,0.40,2.5,23,0.61,3.6
4058,2,0.00,1.1,3,0.00,0.8,6,0.67,3.4,7,...,5.1,5,0.80,6.1,15,0.40,4.3,14,0.57,3.4


In [74]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

      blueTopGP  blueTopWR  blueTopKDA  blueJungleGP  blueJungleWR  \
23            5       0.60         3.7             5          0.60   
1819          0       0.00         0.0             1          0.00   
3712         13       0.38         2.2             1          1.00   
92            3       0.33         1.1             3          0.33   
1977          9       0.56         3.6            11          0.64   
...         ...        ...         ...           ...           ...   
2977         13       0.38         2.0             7          0.43   
2430          3       0.33         4.3             1          0.00   
1805         19       0.63         2.7             5          0.80   
2414          5       0.80         3.8            12          0.50   
2513          2       0.50         2.8             2          0.00   

      blueJungleKDA  blueMidGP  blueMidWR  blueMidKDA  blueADCGP  ...  \
23              9.7          5       0.40         2.9          4  ...   
1819         

In [75]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
C:\Users\linco\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [76]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [82]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
scaler = StandardScaler()    
scaler.fit(X_train)

# FLAMENGO VS PAIN
match_to_predict = {'blueTopGP': [0],'blueTopWR': [0],'blueTopKDA': [0],'blueJungleGP': [5],'blueJungleWR': [0.4],'blueJungleKDA': [4],'blueMidGP': [0],'blueMidWR': [0],'blueMidKDA': [0],'blueADCGP': [2],'blueADCWR': [1],'blueADCKDA': [6.8],'blueSupportGP': [12],'blueSupportWR': [0.42],'blueSupportKDA': [3.1],'redTopGP': [0],'redTopWR': [0],'redTopKDA': [0],'redJungleGP': [10],'redJungleWR': [0.6],'redJungleKDA': [5.7],'redMidGP': [10],'redMidWR': [0.5],'redMidKDA': [2.8],'redAdcGP': [27],'redAdcWR': [0.7],'redAdcKDA': [6],'redSupportGP': [9],'redSupportWR': [0.33],'redSupportKDA': [1.8]}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('A distribuição de probabilidades para FLAMENGO VS PAIN é:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))

# KABUM VS VORAX
match_to_predict = {'blueTopGP': [12],'blueTopWR': [0.5],'blueTopKDA': [2.7],'blueJungleGP': [2],'blueJungleWR': [0.5],'blueJungleKDA': [2.4],'blueMidGP': [0],'blueMidWR': [0],'blueMidKDA': [0],'blueADCGP': [0],'blueADCWR': [0],'blueADCKDA': [0],'blueSupportGP': [2],'blueSupportWR': [0.5],'blueSupportKDA': [15],'redTopGP': [10],'redTopWR': [0.6],'redTopKDA': [4],'redJungleGP': [16],'redJungleWR': [0.63],'redJungleKDA': [3.8],'redMidGP': [23],'redMidWR': [0.38],'redMidKDA': [4.6],'redAdcGP': [6],'redAdcWR': [0.5],'redAdcKDA': [13.2],'redSupportGP': [20],'redSupportWR': [0.75],'redSupportKDA': [5.5]}
dict_match = pd.DataFrame.from_dict(match_to_predict)

X_test = scaler.transform(dict_match)
print('A distribuição de probabilidades para KABUM VS VORAX é:')
print(GBCmodel.predict_proba(X_test))
print(GBCmodel.predict(X_test))


A distribuição de probabilidades para FLAMENGO VS PAIN é:
[[0.35430319 0.64569681]]
[1]
A distribuição de probabilidades para KABUM VS VORAX é:
[[0.16961702 0.83038298]]
[1]


In [78]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.937192,0.929134,0.936508,0.932806
1,Support Vector Machine (Linear Kernel),0.938424,0.929319,0.939153,0.934211
2,Support Vector Machine (RBF Kernel),0.926108,0.918421,0.923280,0.920844
3,Decission Tree,0.876847,0.867725,0.867725,0.867725
4,Adaboost,0.945813,0.941799,0.941799,0.941799
5,Random Forest,0.933498,0.940217,0.915344,0.927614
6,Gradient Boosting Classifier,0.947044,0.939633,0.947090,0.943347
